In [1]:
!unzip /content/train.zip -d /content/train > etc\null

replace /content/train/train/00175.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [0]:
!unzip /content/test.zip -d /content/test > etc\null

replace /content/test/test/0.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [0]:
!unzip /content/verif.zip -d /content/verif > etc\null

In [0]:
import os
import cv2
import torch as t
import torchvision
from PIL import Image
from torch.utils.data import Dataset

PATH_Train = '/content/train/train'
PATH_Test = '/content/test/test'
PATH_Verif = '/content/verif/verif'
end_epoch = 110

# 自定义正确率函数

In [0]:
#正确率函数
def accuracy_rate(pre,tru):
    num = 0
    den = 0
    for x,y in zip(pre,tru):
        den+=1
        if x == y:
            num+=1
    return num/den

# one-hot编码解码函数

In [0]:
import numpy as np


def encode(text):
    text = list(text)
    vector = np.zeros(62 * 5, dtype=float)
    def char2pos(c):
        if c == '_':
            k = 62
            return k
        k = ord(c) - 48
        if k > 9:
            k = ord(c) - 65 + 10
            if k > 35:
                k = ord(c) - 97 + 26 + 10
                if k > 61:
                    raise ValueError('error')
        return k

    for i, c in enumerate(text):
        idx = i * 62 + char2pos(c)
        # print(idx)
        vector[idx] = 1.0
    return vector


import numpy as np
def decode(code):
    code = code.data.cpu().numpy()
    text = []
    for i in range(0,5):
        one_char = code[62*i : 62*(i+1)]
        char_idx = np.argmax(one_char)
        if char_idx < 10:
            char_code = char_idx + ord('0')
        elif char_idx < 36:
            char_code = char_idx - 10 + ord('A')
        elif char_idx < 62:
            char_code = char_idx - 36 + ord('a')
        elif char_idx == 62:
            char_code = ord('_')
        else:
            raise ValueError('error')
        text.append(chr(char_code))
    return "".join(text)


# def pro2code(pro):
#     code = t.zeros(62 * 5)
#     for i in range(0,5):
#         one_char = code[62*i : 62*(i+1)]
#         char_idx = np.argmax(one_char.numpy())
#         code[char_idx + 62*i] = 1.0
#     return code


# 编写验证码类：训练类测试类

In [0]:
from torchvision import transforms as T
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder

class vc_train(Dataset):
    def __init__(self, root, transfroms = None):
        files = os.listdir(root)
        self.img = [image for image in files if image.endswith('.jpg')]
        self.images = [os.path.join(root,image) for image in self.img]
        self.transfroms = transfroms
        
    def __getitem__(self, index):
        image_path = self.images[index]
        label = self.img[index].split('.')[0]
        label = t.Tensor(encode(label))
        pil_image = Image.open(image_path)
        
        if self.transfroms: # 如果有这个参数则这么处理
            data = self.transfroms(pil_image)
        else:
            image_array = np.asarray(pil_image)
            data = torch.from_numpy(image_array)
        return data,label
    
    def __len__(self):
        return len(self.images)

class vc_test(Dataset):
    def __init__(self, root, transfroms = None):
        files = os.listdir(root)
        self.img = [image for image in files if image.endswith('.jpg')]
        self.images = [os.path.join(root,image) for image in self.img]
        self.transfroms = transfroms
        
    def __getitem__(self, index):
        image_path = self.images[index]
        pil_image = Image.open(image_path)
        
        if self.transfroms: # 如果有这个参数则这么处理
            data = self.transfroms(pil_image)
        else:
            image_array = np.asarray(pil_image)
            data = torch.from_numpy(image_array)
        return data
    
    def __len__(self):
        return len(self.images)class vc_test(Dataset):
    def __init__(self, root, transfroms = None):
        files = os.listdir(root)
        self.img = [image for image in files if image.endswith('.jpg')]
        self.images = [os.path.join(root,image) for image in self.img]
        self.transfroms = transfroms
        
    def __getitem__(self, index):
        image_path = self.images[index]
        pil_image = Image.open(image_path)
        
        if self.transfroms: # 如果有这个参数则这么处理
            data = self.transfroms(pil_image)
        else:
            image_array = np.asarray(pil_image)
            data = torch.from_numpy(image_array)
        return data
    
    def __len__(self):
        return len(self.images)

# 加载训练集

In [0]:
transfrom = T.Compose({
    T.ToTensor()
})

dataset = vc_train(PATH_Train,transfroms=transfrom)


dataloader = DataLoader(dataset, batch_size=64, shuffle=True,
                        num_workers=0, drop_last=False)

next(enumerate(dataloader))

# 加载验证集

In [0]:
transfrom = T.Compose({
    T.ToTensor()
})

verifset = vc_train(PATH_Verif,transfroms=transfrom)


verifloader = DataLoader(verifset, batch_size=1024, shuffle=True,
                        num_workers=0, drop_last=True)

verif_idx,(verif_x,verif_y) = next(enumerate(dataloader))

# 继承残差网络34，编写保存加载函数

In [0]:
from torchvision import models
from torch import nn

class RES34(nn.Module):
    def __init__(self):
        super(RES34, self).__init__()
        self.num_cls = 62 * 5
        self.base = torchvision.models.resnet34(pretrained=False)
        self.base.fc = nn.Linear(self.base.fc.in_features, self.num_cls)
    def forward(self, x):
        x = self.base(x)
        return x
    def save(self):
        name = "net_new.pth"
        t.save(self.state_dict(), name)

    def loadIfExist(self):
        fileList = os.listdir("./")
        # print(fileList)
        if "net_new.pth" in fileList:
            name = "net_new.pth"
            self.load_state_dict(t.load(name))
            print("the latest model has been load")


# 实例化残差网络

In [0]:
from torchvision.models import AlexNet
from torch import nn, optim

model = RES34()
model.loadIfExist()
if t.cuda.is_available():
  model = model.cuda()
  
# print(model)

# 设置loss计算函数和优化器，反向传播

In [0]:
from torch.nn.functional import binary_cross_entropy_with_logits
from torch.nn import functional as F
import torch as t
from torch import nn
import numpy as np

criterion = nn.MultiLabelSoftMarginLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
tru = [decode(i) for i in verif_y]

for epoch in range(0, end_epoch):
    for batch_idx,(x,label) in enumerate(dataloader):
        # 前向传播
        x = x.cuda()
        label = label.cuda()
        y_pred = model(x).cuda()

        # 计算损失
        loss = criterion(y_pred,label)

        # 计算正确率 
        verif_x, verif_y = verif_x.cuda(), verif_y.cuda()
        verif_pred = model(x)
        pre = [decode(i) for i in verif_pred]
        train_correct = accuracy_rate(pre,tru)

        
        if batch_idx % 100 == 0:
          print('\n第%d个epoch 第%d索引的loss:'%(epoch+1,batch_idx) ,loss,'验证集正确率：',train_correct)
        
        optimizer.zero_grad()

        #反向传播
        loss.backward()

        optimizer.step()

# 保存网络

In [0]:
 model.save()

# 加载测试集

In [0]:
transfrom = T.Compose({
    T.ToTensor()
})



testset = vc_test(PATH_Test,transfroms=transfrom)


testloader = DataLoader(testset, batch_size=64, shuffle=True,num_workers=0, drop_last=False)

next(enumerate(testloader))

# 加载模型预测测试集

In [0]:
import numpy as np

result = []
model = RES34().cuda()
model.loadIfExist()
for batch_idx,x in enumerate(testloader):
    x = x.cuda()
    y_pred = model(x)
    pred = [decode(i) for i in y_pred]
    result += pred

# 保存并查看结果

In [0]:
import pandas as pd

id_y = np.arange(20000).reshape((-1,1))

# 将预测结果制作成DataFrame形式便于查看
result_df = pd.DataFrame(id_y,columns=['id'],dtype='int32')

result_df['y'] = pd.Series(result)

result_df.to_csv('submit_test.csv',index=False, sep=',')


# 输出预测结果
print('预测结果：')
result_df
